# BetaBae Results Exploration

This notebook explores the training results from the minimal BetaBae agent.

## What to Look For:
- **Attention Evolution**: Does the attention matrix show structure forming over time?
- **Representation Learning**: Are hidden states organizing into meaningful clusters?
- **Learning Dynamics**: How do prediction error and action entropy co-evolve?
- **Emergence**: Can we detect when structure emerges in the agent's internal representations?


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Load data
log_dir = Path('logs/run_001')
episodes = sorted(log_dir.glob('episode_*.npz'))

print(f"Found {len(episodes)} episodes")
print(f"Episode files: {[f.name for f in episodes[:5]]}...")


In [ ]:
# Collect all metrics across episodes
all_pred_errors = []
all_actions = []
episode_lengths = []

for ep_file in episodes:
    data = np.load(ep_file)
    all_pred_errors.extend(data['pred_error'])
    all_actions.extend(data['actions'])
    episode_lengths.append(len(data['pred_error']))

# Plot learning curves
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Prediction error over time
axes[0, 0].plot(all_pred_errors, alpha=0.7, linewidth=0.5)
axes[0, 0].set_title('Prediction Error Evolution')
axes[0, 0].set_xlabel('Training Step')
axes[0, 0].set_ylabel('MSE Loss')
axes[0, 0].grid(True, alpha=0.3)

# Episode lengths (performance proxy)
axes[0, 1].plot(episode_lengths, marker='o', markersize=3)
axes[0, 1].set_title('Episode Lengths (Performance)')
axes[0, 1].set_xlabel('Episode')
axes[0, 1].set_ylabel('Steps')
axes[0, 1].grid(True, alpha=0.3)

# Action distribution over time
action_counts = np.bincount(all_actions)
axes[1, 0].bar(range(len(action_counts)), action_counts, alpha=0.7)
axes[1, 0].set_title('Action Distribution')
axes[1, 0].set_xlabel('Action')
axes[1, 0].set_ylabel('Count')

# Rolling average of prediction error
window = 50
if len(all_pred_errors) > window:
    rolling_avg = np.convolve(all_pred_errors, np.ones(window)/window, mode='valid')
    axes[1, 1].plot(rolling_avg, linewidth=2)
    axes[1, 1].set_title(f'Rolling Average Prediction Error (window={window})')
    axes[1, 1].set_xlabel('Training Step')
    axes[1, 1].set_ylabel('MSE Loss')
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nSummary Statistics:")
print(f"Total training steps: {len(all_pred_errors)}")
print(f"Average episode length: {np.mean(episode_lengths):.1f} steps")
print(f"Final prediction error: {all_pred_errors[-1]:.4f}")
print(f"Initial prediction error: {all_pred_errors[0]:.4f}")
